# 08. 결과 시각화 및 리포트 생성 기능

**담당**: Claude Opus  
**작성일**: 2025-10-24  
**목적**: 드론 작물 탐지 결과를 시각화하고 리포트를 생성하는 시스템

## 주요 기능
- 탐지 결과 시각화 (차트, 히트맵)
- PDF/HTML 리포트 자동 생성
- 대시보드 인터페이스
- 지리공간 시각화
- 시계열 분석 차트

In [ ]:
# 필요 패키지 설치
import sys
!{sys.executable} -m pip install matplotlib seaborn plotly folium reportlab jinja2 pandas numpy opencv-python-headless

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from folium import plugins
import cv2
from PIL import Image, ImageDraw, ImageFont
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image as RLImage, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_RIGHT
from reportlab.pdfgen import canvas
from jinja2 import Template
import base64
from io import BytesIO
import logging
from collections import defaultdict

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Matplotlib 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# 한글 폰트 설정 (필요시)
# plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 데이터 준비 및 통계

In [ ]:
@dataclass
class VisualizationData:
    """시각화용 데이터 구조"""
    detection_results: pd.DataFrame
    summary_stats: Dict
    temporal_data: pd.DataFrame
    spatial_data: List[Dict]
    class_distribution: Dict
    confidence_distribution: List[float]

class DataPreparator:
    """시각화를 위한 데이터 준비"""
    
    @staticmethod
    def prepare_visualization_data(detection_data: List[Dict]) -> VisualizationData:
        """시각화 데이터 준비"""
        
        # DataFrame 변환
        df = pd.DataFrame(detection_data)
        
        # 시간 데이터가 있는 경우 datetime으로 변환
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df['date'] = df['timestamp'].dt.date
            df['hour'] = df['timestamp'].dt.hour
        
        # 요약 통계
        summary_stats = {
            'total_images': df['image_id'].nunique() if 'image_id' in df.columns else 0,
            'total_detections': len(df),
            'avg_confidence': df['confidence'].mean() if 'confidence' in df.columns else 0,
            'unique_classes': df['class_name'].nunique() if 'class_name' in df.columns else 0
        }
        
        # 시간별 데이터
        temporal_data = pd.DataFrame()
        if 'timestamp' in df.columns:
            temporal_data = df.groupby('date').agg({
                'image_id': 'nunique',
                'class_name': 'count'
            }).rename(columns={'image_id': 'images', 'class_name': 'detections'})
        
        # 공간 데이터
        spatial_data = []
        if all(col in df.columns for col in ['gps_latitude', 'gps_longitude']):
            for _, row in df.iterrows():
                if pd.notna(row.get('gps_latitude')) and pd.notna(row.get('gps_longitude')):
                    spatial_data.append({
                        'lat': row['gps_latitude'],
                        'lon': row['gps_longitude'],
                        'class': row.get('class_name', 'unknown'),
                        'confidence': row.get('confidence', 0)
                    })
        
        # 클래스 분포
        class_distribution = {}
        if 'class_name' in df.columns:
            class_distribution = df['class_name'].value_counts().to_dict()
        
        # 신뢰도 분포
        confidence_distribution = []
        if 'confidence' in df.columns:
            confidence_distribution = df['confidence'].tolist()
        
        return VisualizationData(
            detection_results=df,
            summary_stats=summary_stats,
            temporal_data=temporal_data,
            spatial_data=spatial_data,
            class_distribution=class_distribution,
            confidence_distribution=confidence_distribution
        )

## 2. 차트 생성

In [ ]:
class ChartGenerator:
    """차트 생성 클래스"""
    
    @staticmethod
    def create_class_distribution_chart(data: VisualizationData, save_path: Optional[str] = None):
        """클래스 분포 차트 생성"""
        if not data.class_distribution:
            return None
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # 막대 그래프
        classes = list(data.class_distribution.keys())
        counts = list(data.class_distribution.values())
        
        bars = ax1.bar(classes, counts, color=sns.color_palette("husl", len(classes)))
        ax1.set_xlabel('작물 클래스', fontsize=12)
        ax1.set_ylabel('탐지 수', fontsize=12)
        ax1.set_title('작물별 탐지 수', fontsize=14, fontweight='bold')
        ax1.tick_params(axis='x', rotation=45)
        
        # 값 표시
        for bar, count in zip(bars, counts):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    str(count), ha='center', va='bottom')
        
        # 파이 차트
        wedges, texts, autotexts = ax2.pie(counts, labels=classes, autopct='%1.1f%%',
                                            colors=sns.color_palette("husl", len(classes)))
        ax2.set_title('작물 분포 비율', fontsize=14, fontweight='bold')
        
        # 텍스트 스타일 개선
        for text in texts:
            text.set_fontsize(10)
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontsize(10)
            autotext.set_fontweight('bold')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            logger.info(f"차트 저장: {save_path}")
        
        return fig
    
    @staticmethod
    def create_confidence_histogram(data: VisualizationData, save_path: Optional[str] = None):
        """신뢰도 히스토그램 생성"""
        if not data.confidence_distribution:
            return None
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # 히스토그램
        n, bins, patches = ax.hist(data.confidence_distribution, bins=20, 
                                   color='skyblue', edgecolor='black', alpha=0.7)
        
        # 평균선
        mean_conf = np.mean(data.confidence_distribution)
        ax.axvline(mean_conf, color='red', linestyle='--', linewidth=2, 
                  label=f'평균: {mean_conf:.2f}')
        
        ax.set_xlabel('신뢰도', fontsize=12)
        ax.set_ylabel('빈도', fontsize=12)
        ax.set_title('탐지 신뢰도 분포', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        return fig
    
    @staticmethod
    def create_temporal_chart(data: VisualizationData, save_path: Optional[str] = None):
        """시계열 차트 생성"""
        if data.temporal_data.empty:
            return None
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # 일별 이미지 수
        ax1.plot(data.temporal_data.index, data.temporal_data['images'], 
                marker='o', linewidth=2, markersize=8, color='blue')
        ax1.fill_between(data.temporal_data.index, data.temporal_data['images'], 
                        alpha=0.3, color='blue')
        ax1.set_ylabel('이미지 수', fontsize=12)
        ax1.set_title('일별 처리 현황', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        # 일별 탐지 수
        ax2.bar(data.temporal_data.index, data.temporal_data['detections'], 
               color='green', alpha=0.7)
        ax2.set_xlabel('날짜', fontsize=12)
        ax2.set_ylabel('탐지 수', fontsize=12)
        ax2.set_title('일별 탐지 수', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # X축 포맷
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        return fig

## 3. 인터랙티브 차트 (Plotly)

In [ ]:
class InteractiveChartGenerator:
    """인터랙티브 차트 생성"""
    
    @staticmethod
    def create_interactive_dashboard(data: VisualizationData) -> go.Figure:
        """인터랙티브 대시보드 생성"""
        
        # 서브플롯 생성
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('클래스 분포', '신뢰도 분포', '시간별 탐지', '상위 탐지 클래스'),
            specs=[[{'type': 'bar'}, {'type': 'histogram'}],
                  [{'type': 'scatter'}, {'type': 'pie'}]]
        )
        
        # 1. 클래스 분포 막대 그래프
        if data.class_distribution:
            fig.add_trace(
                go.Bar(
                    x=list(data.class_distribution.keys()),
                    y=list(data.class_distribution.values()),
                    marker_color='lightblue',
                    text=list(data.class_distribution.values()),
                    textposition='auto'
                ),
                row=1, col=1
            )
        
        # 2. 신뢰도 히스토그램
        if data.confidence_distribution:
            fig.add_trace(
                go.Histogram(
                    x=data.confidence_distribution,
                    nbinsx=20,
                    marker_color='lightgreen'
                ),
                row=1, col=2
            )
        
        # 3. 시간별 탐지 (시계열)
        if not data.temporal_data.empty:
            fig.add_trace(
                go.Scatter(
                    x=data.temporal_data.index,
                    y=data.temporal_data['detections'],
                    mode='lines+markers',
                    marker=dict(size=8),
                    line=dict(width=2)
                ),
                row=2, col=1
            )
        
        # 4. 상위 5개 클래스 파이 차트
        if data.class_distribution:
            top_classes = dict(sorted(data.class_distribution.items(), 
                                    key=lambda x: x[1], reverse=True)[:5])
            fig.add_trace(
                go.Pie(
                    labels=list(top_classes.keys()),
                    values=list(top_classes.values()),
                    hole=0.3
                ),
                row=2, col=2
            )
        
        # 레이아웃 업데이트
        fig.update_layout(
            height=800,
            showlegend=False,
            title_text="드론 작물 탐지 대시보드",
            title_font_size=20
        )
        
        # 축 레이블 업데이트
        fig.update_xaxes(title_text="클래스", row=1, col=1)
        fig.update_yaxes(title_text="탐지 수", row=1, col=1)
        fig.update_xaxes(title_text="신뢰도", row=1, col=2)
        fig.update_yaxes(title_text="빈도", row=1, col=2)
        fig.update_xaxes(title_text="날짜", row=2, col=1)
        fig.update_yaxes(title_text="탐지 수", row=2, col=1)
        
        return fig
    
    @staticmethod
    def create_3d_scatter(data: VisualizationData) -> go.Figure:
        """3D 산점도 생성"""
        df = data.detection_results
        
        if not all(col in df.columns for col in ['x1', 'y1', 'confidence']):
            return None
        
        fig = go.Figure(data=[go.Scatter3d(
            x=df['x1'],
            y=df['y1'],
            z=df['confidence'],
            mode='markers',
            marker=dict(
                size=5,
                color=df['confidence'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="신뢰도")
            ),
            text=df['class_name'] if 'class_name' in df.columns else None,
            hovertemplate='<b>%{text}</b><br>' +
                         'X: %{x}<br>' +
                         'Y: %{y}<br>' +
                         '신뢰도: %{z:.2f}<br>'
        )])
        
        fig.update_layout(
            scene=dict(
                xaxis_title='X 좌표',
                yaxis_title='Y 좌표',
                zaxis_title='신뢰도'
            ),
            title="탐지 결과 3D 분포",
            height=600
        )
        
        return fig

## 4. 지리공간 시각화

In [ ]:
class GeoVisualization:
    """지리공간 시각화"""
    
    @staticmethod
    def create_detection_map(data: VisualizationData, save_path: Optional[str] = None) -> folium.Map:
        """탐지 결과 지도 생성"""
        
        if not data.spatial_data:
            logger.warning("GPS 데이터가 없습니다")
            return None
        
        # 중심점 계산
        center_lat = np.mean([d['lat'] for d in data.spatial_data])
        center_lon = np.mean([d['lon'] for d in data.spatial_data])
        
        # 지도 생성
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=15,
            tiles='OpenStreetMap'
        )
        
        # 위성 이미지 레이어 추가
        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='위성 이미지',
            overlay=False,
            control=True
        ).add_to(m)
        
        # 클래스별 색상 정의
        class_colors = {
            'wheat': 'orange',
            'corn': 'yellow',
            'rice': 'green',
            'soybean': 'brown',
            'disease': 'red',
            'pest': 'purple'
        }
        
        # 클래스별 FeatureGroup 생성
        feature_groups = {}
        for class_name in set(d['class'] for d in data.spatial_data):
            fg = folium.FeatureGroup(name=class_name)
            feature_groups[class_name] = fg
            m.add_child(fg)
        
        # 마커 추가
        for point in data.spatial_data:
            color = class_colors.get(point['class'], 'blue')
            
            folium.CircleMarker(
                location=[point['lat'], point['lon']],
                radius=5 + point['confidence'] * 10,  # 신뢰도에 따른 크기
                popup=f"클래스: {point['class']}<br>신뢰도: {point['confidence']:.2f}",
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.6
            ).add_to(feature_groups[point['class']])
        
        # 히트맵 레이어 추가
        heat_data = [[d['lat'], d['lon'], d['confidence']] for d in data.spatial_data]
        plugins.HeatMap(heat_data, name="히트맵").add_to(m)
        
        # 클러스터 마커 추가
        marker_cluster = plugins.MarkerCluster(name="클러스터")
        for point in data.spatial_data:
            folium.Marker(
                location=[point['lat'], point['lon']],
                popup=f"{point['class']}: {point['confidence']:.2f}"
            ).add_to(marker_cluster)
        marker_cluster.add_to(m)
        
        # 레이어 컨트롤 추가
        folium.LayerControl().add_to(m)
        
        # 미니맵 추가
        minimap = plugins.MiniMap()
        m.add_child(minimap)
        
        # 저장
        if save_path:
            m.save(save_path)
            logger.info(f"지도 저장: {save_path}")
        
        return m
    
    @staticmethod
    def create_field_heatmap(data: VisualizationData, field_boundary: List[Tuple[float, float]] = None):
        """농장 필드 히트맵 생성"""
        
        if not data.spatial_data:
            return None
        
        # 데이터를 그리드로 변환
        lats = [d['lat'] for d in data.spatial_data]
        lons = [d['lon'] for d in data.spatial_data]
        values = [d['confidence'] for d in data.spatial_data]
        
        # 그리드 생성
        lat_range = np.linspace(min(lats), max(lats), 50)
        lon_range = np.linspace(min(lons), max(lons), 50)
        
        # 히트맵 데이터 생성 (간단한 보간)
        heatmap_data = np.zeros((len(lat_range), len(lon_range)))
        
        for lat, lon, val in zip(lats, lons, values):
            lat_idx = np.argmin(np.abs(lat_range - lat))
            lon_idx = np.argmin(np.abs(lon_range - lon))
            heatmap_data[lat_idx, lon_idx] = max(heatmap_data[lat_idx, lon_idx], val)
        
        # Plotly 히트맵
        fig = go.Figure(data=go.Heatmap(
            z=heatmap_data,
            x=lon_range,
            y=lat_range,
            colorscale='RdYlGn',
            reversescale=True,
            colorbar=dict(title="탐지 강도")
        ))
        
        # 필드 경계 추가
        if field_boundary:
            boundary_lats = [b[0] for b in field_boundary]
            boundary_lons = [b[1] for b in field_boundary]
            fig.add_trace(go.Scatter(
                x=boundary_lons + [boundary_lons[0]],
                y=boundary_lats + [boundary_lats[0]],
                mode='lines',
                line=dict(color='black', width=2),
                name='필드 경계'
            ))
        
        fig.update_layout(
            title="농장 필드 탐지 히트맵",
            xaxis_title="경도",
            yaxis_title="위도",
            height=600
        )
        
        return fig

## 5. 리포트 생성

In [ ]:
class ReportGenerator:
    """리포트 생성 클래스"""
    
    def __init__(self, data: VisualizationData):
        self.data = data
        self.styles = getSampleStyleSheet()
        self._setup_styles()
        
    def _setup_styles(self):
        """스타일 설정"""
        # 제목 스타일
        self.styles.add(ParagraphStyle(
            name='CustomTitle',
            parent=self.styles['Heading1'],
            fontSize=24,
            textColor=colors.HexColor('#1f77b4'),
            spaceAfter=30,
            alignment=TA_CENTER
        ))
        
        # 부제목 스타일
        self.styles.add(ParagraphStyle(
            name='CustomHeading',
            parent=self.styles['Heading2'],
            fontSize=16,
            textColor=colors.HexColor('#2ca02c'),
            spaceAfter=12
        ))
    
    def generate_pdf_report(self, output_path: str, include_images: List[str] = None):
        """PDF 리포트 생성"""
        
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=18
        )
        
        story = []
        
        # 제목
        title = Paragraph("드론 작물 탐지 분석 리포트", self.styles['CustomTitle'])
        story.append(title)
        story.append(Spacer(1, 12))
        
        # 생성 정보
        date_text = Paragraph(f"생성일: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 
                             self.styles['Normal'])
        story.append(date_text)
        story.append(Spacer(1, 20))
        
        # 요약 통계 섹션
        story.append(Paragraph("1. 요약 통계", self.styles['CustomHeading']))
        
        stats_data = [
            ['항목', '값'],
            ['총 이미지 수', str(self.data.summary_stats['total_images'])],
            ['총 탐지 수', str(self.data.summary_stats['total_detections'])],
            ['평균 신뢰도', f"{self.data.summary_stats['avg_confidence']:.2%}"],
            ['탐지 클래스 수', str(self.data.summary_stats['unique_classes'])]
        ]
        
        stats_table = Table(stats_data, colWidths=[3*inch, 2*inch])
        stats_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 12),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black)
        ]))
        story.append(stats_table)
        story.append(Spacer(1, 20))
        
        # 클래스 분포 섹션
        story.append(Paragraph("2. 클래스별 탐지 분포", self.styles['CustomHeading']))
        
        if self.data.class_distribution:
            class_data = [['클래스', '탐지 수', '비율']]
            total = sum(self.data.class_distribution.values())
            
            for class_name, count in sorted(self.data.class_distribution.items(), 
                                           key=lambda x: x[1], reverse=True):
                percentage = (count / total) * 100
                class_data.append([class_name, str(count), f"{percentage:.1f}%"])
            
            class_table = Table(class_data, colWidths=[2*inch, 1.5*inch, 1.5*inch])
            class_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 11),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.lightgrey),
                ('GRID', (0, 0), (-1, -1), 1, colors.black)
            ]))
            story.append(class_table)
        
        story.append(PageBreak())
        
        # 이미지 추가
        if include_images:
            story.append(Paragraph("3. 분석 차트", self.styles['CustomHeading']))
            
            for img_path in include_images:
                if os.path.exists(img_path):
                    img = RLImage(img_path, width=5*inch, height=3*inch)
                    story.append(img)
                    story.append(Spacer(1, 12))
        
        # 권장사항
        story.append(Paragraph("4. 권장사항", self.styles['CustomHeading']))
        
        recommendations = self._generate_recommendations()
        for rec in recommendations:
            story.append(Paragraph(f"• {rec}", self.styles['Normal']))
            story.append(Spacer(1, 6))
        
        # PDF 생성
        doc.build(story)
        logger.info(f"PDF 리포트 생성: {output_path}")
    
    def _generate_recommendations(self) -> List[str]:
        """권장사항 생성"""
        recommendations = []
        
        # 신뢰도 기반 권장사항
        avg_conf = self.data.summary_stats['avg_confidence']
        if avg_conf < 0.7:
            recommendations.append("평균 신뢰도가 낮습니다. 모델 재훈련을 고려하세요.")
        
        # 클래스 분포 기반
        if self.data.class_distribution:
            if 'disease' in self.data.class_distribution:
                disease_count = self.data.class_distribution['disease']
                if disease_count > 10:
                    recommendations.append(f"병해 탐지 {disease_count}건 발견. 즉시 조치가 필요합니다.")
            
            if 'pest' in self.data.class_distribution:
                pest_count = self.data.class_distribution['pest']
                if pest_count > 5:
                    recommendations.append(f"해충 탐지 {pest_count}건 발견. 방제 조치를 검토하세요.")
        
        # 시간별 분석
        if not self.data.temporal_data.empty:
            recent_trend = self.data.temporal_data.tail(3)['detections'].mean()
            if recent_trend > self.data.temporal_data['detections'].mean() * 1.5:
                recommendations.append("최근 탐지 수가 증가 추세입니다. 집중 모니터링이 필요합니다.")
        
        if not recommendations:
            recommendations.append("현재 특별한 조치가 필요하지 않습니다.")
            recommendations.append("정기적인 모니터링을 계속 진행하세요.")
        
        return recommendations
    
    def generate_html_report(self, output_path: str, charts: List[go.Figure] = None):
        """HTML 리포트 생성"""
        
        template_str = '''
        <!DOCTYPE html>
        <html>
        <head>
            <title>드론 작물 탐지 리포트</title>
            <meta charset="utf-8">
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }
                h1 { color: #1f77b4; text-align: center; }
                h2 { color: #2ca02c; border-bottom: 2px solid #2ca02c; padding-bottom: 5px; }
                .stats-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 20px; margin: 20px 0; }
                .stat-card { background: white; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
                .stat-value { font-size: 28px; font-weight: bold; color: #333; }
                .stat-label { color: #666; margin-top: 5px; }
                table { width: 100%; border-collapse: collapse; background: white; margin: 20px 0; }
                th { background: #1f77b4; color: white; padding: 12px; text-align: left; }
                td { padding: 10px; border-bottom: 1px solid #ddd; }
                .chart-container { background: white; padding: 20px; margin: 20px 0; border-radius: 8px; }
                .recommendations { background: #fff3cd; padding: 20px; border-radius: 8px; margin: 20px 0; }
                .footer { text-align: center; color: #666; margin-top: 40px; padding-top: 20px; border-top: 1px solid #ddd; }
            </style>
        </head>
        <body>
            <h1>드론 작물 탐지 분석 리포트</h1>
            <p style="text-align: center; color: #666;">생성일: {{ generation_date }}</p>
            
            <h2>요약 통계</h2>
            <div class="stats-grid">
                <div class="stat-card">
                    <div class="stat-value">{{ total_images }}</div>
                    <div class="stat-label">총 이미지</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{{ total_detections }}</div>
                    <div class="stat-label">총 탐지</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{{ avg_confidence }}%</div>
                    <div class="stat-label">평균 신뢰도</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{{ unique_classes }}</div>
                    <div class="stat-label">클래스 수</div>
                </div>
            </div>
            
            <h2>클래스별 분포</h2>
            <table>
                <tr>
                    <th>클래스</th>
                    <th>탐지 수</th>
                    <th>비율</th>
                </tr>
                {{ class_table }}
            </table>
            
            {{ charts_html }}
            
            <h2>권장사항</h2>
            <div class="recommendations">
                {{ recommendations }}
            </div>
            
            <div class="footer">
                <p>© 2025 드론 작물 모니터링 시스템</p>
            </div>
        </body>
        </html>
        '''
        
        template = Template(template_str)
        
        # 클래스 테이블 생성
        class_table_html = ""
        if self.data.class_distribution:
            total = sum(self.data.class_distribution.values())
            for class_name, count in sorted(self.data.class_distribution.items(), 
                                           key=lambda x: x[1], reverse=True):
                percentage = (count / total) * 100
                class_table_html += f"<tr><td>{class_name}</td><td>{count}</td><td>{percentage:.1f}%</td></tr>"
        
        # 차트 HTML
        charts_html = ""
        if charts:
            charts_html = "<h2>분석 차트</h2>"
            for chart in charts:
                charts_html += f'<div class="chart-container">{chart.to_html(include_plotlyjs="cdn")}</div>'
        
        # 권장사항
        recommendations_html = "<ul>"
        for rec in self._generate_recommendations():
            recommendations_html += f"<li>{rec}</li>"
        recommendations_html += "</ul>"
        
        # HTML 생성
        html_content = template.render(
            generation_date=datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            total_images=self.data.summary_stats['total_images'],
            total_detections=self.data.summary_stats['total_detections'],
            avg_confidence=round(self.data.summary_stats['avg_confidence'] * 100, 1),
            unique_classes=self.data.summary_stats['unique_classes'],
            class_table=class_table_html,
            charts_html=charts_html,
            recommendations=recommendations_html
        )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        logger.info(f"HTML 리포트 생성: {output_path}")

## 6. 통합 사용 예제

In [ ]:
def demo_visualization_system():
    """시각화 시스템 데모"""
    
    print("🎨 시각화 및 리포트 생성 시스템 데모\n")
    
    # 1. 테스트 데이터 생성
    print("[1/5] 테스트 데이터 생성...")
    
    # 가상의 탐지 데이터 생성
    test_data = []
    classes = ['wheat', 'corn', 'rice', 'soybean', 'disease', 'pest']
    
    for i in range(100):
        test_data.append({
            'image_id': f'img_{i:04d}',
            'class_name': np.random.choice(classes, p=[0.3, 0.25, 0.2, 0.15, 0.07, 0.03]),
            'confidence': np.random.uniform(0.6, 0.99),
            'x1': np.random.randint(0, 500),
            'y1': np.random.randint(0, 500),
            'x2': np.random.randint(100, 640),
            'y2': np.random.randint(100, 640),
            'timestamp': datetime.now() - timedelta(days=np.random.randint(0, 30)),
            'gps_latitude': 37.5 + np.random.uniform(-0.01, 0.01),
            'gps_longitude': 127.0 + np.random.uniform(-0.01, 0.01)
        })
    
    # 2. 데이터 준비
    print("[2/5] 데이터 준비...")
    data = DataPreparator.prepare_visualization_data(test_data)
    
    print(f"  - 총 탐지: {data.summary_stats['total_detections']}")
    print(f"  - 평균 신뢰도: {data.summary_stats['avg_confidence']:.2%}")
    print(f"  - 클래스 수: {data.summary_stats['unique_classes']}")
    
    # 3. 차트 생성
    print("\n[3/5] 차트 생성...")
    
    # 정적 차트
    chart_gen = ChartGenerator()
    
    # 출력 디렉토리 생성
    output_dir = Path("visualization_output")
    output_dir.mkdir(exist_ok=True)
    
    # 클래스 분포 차트
    fig1 = chart_gen.create_class_distribution_chart(
        data, 
        save_path=str(output_dir / "class_distribution.png")
    )
    plt.close(fig1)
    print("  ✓ 클래스 분포 차트 생성")
    
    # 신뢰도 히스토그램
    fig2 = chart_gen.create_confidence_histogram(
        data,
        save_path=str(output_dir / "confidence_histogram.png")
    )
    plt.close(fig2)
    print("  ✓ 신뢰도 히스토그램 생성")
    
    # 시계열 차트
    fig3 = chart_gen.create_temporal_chart(
        data,
        save_path=str(output_dir / "temporal_chart.png")
    )
    if fig3:
        plt.close(fig3)
        print("  ✓ 시계열 차트 생성")
    
    # 4. 인터랙티브 차트
    print("\n[4/5] 인터랙티브 차트 생성...")
    
    interactive_gen = InteractiveChartGenerator()
    
    # 대시보드
    dashboard = interactive_gen.create_interactive_dashboard(data)
    dashboard.write_html(str(output_dir / "dashboard.html"))
    print("  ✓ 인터랙티브 대시보드 생성")
    
    # 3D 산점도
    scatter_3d = interactive_gen.create_3d_scatter(data)
    if scatter_3d:
        scatter_3d.write_html(str(output_dir / "3d_scatter.html"))
        print("  ✓ 3D 산점도 생성")
    
    # 지도
    geo_viz = GeoVisualization()
    map_obj = geo_viz.create_detection_map(
        data,
        save_path=str(output_dir / "detection_map.html")
    )
    if map_obj:
        print("  ✓ 지리공간 지도 생성")
    
    # 5. 리포트 생성
    print("\n[5/5] 리포트 생성...")
    
    report_gen = ReportGenerator(data)
    
    # PDF 리포트
    report_gen.generate_pdf_report(
        str(output_dir / "analysis_report.pdf"),
        include_images=[
            str(output_dir / "class_distribution.png"),
            str(output_dir / "confidence_histogram.png")
        ]
    )
    print("  ✓ PDF 리포트 생성")
    
    # HTML 리포트
    report_gen.generate_html_report(
        str(output_dir / "analysis_report.html"),
        charts=[dashboard]
    )
    print("  ✓ HTML 리포트 생성")
    
    print(f"\n✅ 모든 시각화 파일이 {output_dir} 폴더에 저장되었습니다!")
    
    return data

# 데모 실행
demo_data = demo_visualization_system()

## 7. 실시간 대시보드 (선택적)

In [ ]:
print("\n📊 실시간 대시보드 기능\n")

print("실시간 대시보드를 위한 추가 기능:")
print("1. Dash/Streamlit 웹 애플리케이션")
print("2. WebSocket 기반 실시간 업데이트")
print("3. RESTful API 엔드포인트")
print("4. 데이터베이스 연동")

print("\n이러한 기능들은 별도의 웹 서버 환경에서 구현됩니다.")

## 8. 전체 시스템 통합

In [ ]:
print("\n🔗 전체 시스템 통합 완료!\n")

print("=" * 60)
print("YOLO11 드론 작물 탐지 시스템 - Opus 담당 모듈 완성")
print("=" * 60)

print("\n✅ 구현 완료된 모듈:")
print("\n1. Todo 2: 드론 영상/이미지 입력 처리 모듈")
print("   - 다양한 포맷 지원 (이미지, 비디오, 스트림)")
print("   - 드론 메타데이터 추출")
print("   - 전처리 및 향상")

print("\n2. Todo 5: 배치 이미지 처리 시스템")
print("   - 멀티프로세싱 병렬 처리")
print("   - 우선순위 큐 관리")
print("   - 동적 리소스 관리")

print("\n3. Todo 6: 탐지 결과 저장 및 관리 시스템")
print("   - SQLite 데이터베이스")
print("   - 다양한 형식 내보내기")
print("   - 지리공간 분석")

print("\n4. Todo 7: 지속적 모니터링 스케줄링 시스템")
print("   - 크론 기반 스케줄링")
print("   - 폴더 실시간 모니터링")
print("   - 알림 시스템")

print("\n5. Todo 8: 결과 시각화 및 리포트 생성")
print("   - 다양한 차트 및 그래프")
print("   - 인터랙티브 대시보드")
print("   - PDF/HTML 리포트 자동 생성")
print("   - 지리공간 시각화")

print("\n" + "="*60)
print("시스템 통합 특징:")
print("- 모듈 간 원활한 데이터 흐름")
print("- 확장 가능한 아키텍처")
print("- 실시간 처리 및 배치 처리 지원")
print("- 포괄적인 모니터링 및 리포팅")

print("\n🎉 Opus 담당 개발 완료! Sonnet과의 협업으로 완전한 시스템 구축!")